In [32]:
import openai
import cv2
import numpy as np
from PIL import Image,ImageDraw
import pytesseract
from collections import Counter
import torch
from transformers import DetrForObjectDetection, DetrImageProcessor
from sklearn.cluster import KMeans
from transformers import CLIPProcessor, CLIPModel
import os
import glob

In [8]:
pip install timm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 7.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.6 MB/s eta 0:00:0000:01:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [9]:
!pip install yolov5

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.4/953.4 kB 11.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/54.8 MB 2.5 MB/s eta 0:00:14^C
   ━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/54.8 MB 2.5 MB/s eta 0:00:14
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [5]:
pip install pytesseract opencv-python

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Load the image
img_path = 'test.png'
image = cv2.imread(img_path)

# Convert the image to HSV color space
hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define the HSV range for blue color
lower_blue = np.array([100, 150, 0])
upper_blue = np.array([140, 255, 255])

# Create a mask for blue regions
blue_mask = cv2.inRange(hsv_img, lower_blue, upper_blue)

# Find contours of the blue regions
contours, _ = cv2.findContours(blue_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Iterate through contours and get the blue pin's coordinates
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    pin_coordinates = (x + w//2, y + h//2)  # Center of the bounding box
    print(f"Blue pin location: {pin_coordinates}")
    break  # Assuming we only need the first largest blue object

Blue pin location: (540, 1092)


In [30]:
# Function to detect pins on the map with restricted pin types
def detect_pins(map_screenshot_path, template_folder_path, threshold_dict, restricted_pins, scale_factors=[1.0]):
    # Load the map screenshot
    map_screenshot = cv2.imread(map_screenshot_path)
    if map_screenshot is None:
        raise ValueError(f"Could not load map screenshot from path: {map_screenshot_path}")
    
    height, width, _ = map_screenshot.shape

    # Define the ROI as the center region (adjust ROI size as needed)
    roi_x_start = width // 4  # Start x at 25% of the width
    roi_x_end = 3 * width // 4  # End x at 75% of the width
    roi_y_start = height // 4  # Start y at 25% of the height
    roi_y_end = 3 * height // 4  # End y at 75% of the height

    # Extract the center ROI
    roi = map_screenshot[roi_y_start:roi_y_end, roi_x_start:roi_x_end]

    # Variable to hold the highest confidence and corresponding details
    best_detection = None

    # Loop through all pin templates in the folder
    for template_name in os.listdir(template_folder_path):
        # Check if this pin type is in the restricted list
        if template_name in restricted_pins:
            print(f"Skipping {template_name}: This pin type is restricted.")
            continue
        
        # Ensure we're only processing image files (filter non-image files)
        if not template_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        # Load the template
        template_path = os.path.join(template_folder_path, template_name)
        template = cv2.imread(template_path)

        if template is None:
            print(f"Could not load template: {template_path}")
            continue

        # Skip matching if the template is larger than the ROI
        if template.shape[0] > roi.shape[0] or template.shape[1] > roi.shape[1]:
            print(f"Skipping {template_name}: Template is larger than the ROI.")
            continue

        # Get the matching threshold for the current pin type
        threshold = threshold_dict.get(template_name, 0.5)  # Default threshold is 0.5

        # Loop through the scale factors for scale-invariant matching
        for scale in scale_factors:
            # Resize the template for each scale
            scaled_template = cv2.resize(template, (0, 0), fx=scale, fy=scale)

            # Skip matching if the resized template is larger than the ROI
            if scaled_template.shape[0] > roi.shape[0] or scaled_template.shape[1] > roi.shape[1]:
                print(f"Skipping scaled {template_name}: Resized template is larger than the ROI.")
                continue

            # Perform template matching within the ROI (center of the image)
            result = cv2.matchTemplate(roi, scaled_template, cv2.TM_CCOEFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

            # If max_val is above the threshold, a marker is detected
            if max_val >= threshold:
                # Adjust max_loc to reflect coordinates in the original image
                pin_x = max_loc[0] + roi_x_start
                pin_y = max_loc[1] + roi_y_start

                # Check if this is the best detection so far
                if best_detection is None or max_val > best_detection['confidence']:
                    best_detection = {
                        'pin_type': template_name,
                        'coordinates': (pin_x, pin_y),
                        'confidence': max_val,
                        'scale': scale
                    }
                break  # Break after the first match for this template
    
    return best_detection

# Define paths to your images and templates
map_screenshot_path = 'zomato.png'  # Change this to your actual map screenshot path
template_folder_path = 'pin_templates/'  # Folder containing pin images (red, green, blue, etc.)

# Define a dictionary with custom thresholds for different pin templates
threshold_dict = {
    'red_pin.png': 0.5,  # Threshold for red pin
    'green_pin.png': 0.5,  # Threshold for green pin
}

# Define a list of restricted pins (pin types you want to skip)
restricted_pins = ['restricted.png']  # Add the names of pins to restrict

# Define scale factors to try (in case pin sizes vary)
scale_factors = [0.8, 1.0, 1.2]

# Detect pins while ignoring restricted pins
best_detection = detect_pins(map_screenshot_path, template_folder_path, threshold_dict, restricted_pins, scale_factors)

# Output the best detection if found
if best_detection:
    print(f"Marker '{best_detection['pin_type']}' detected at coordinates {best_detection['coordinates']} with confidence {best_detection['confidence']} at scale {best_detection['scale']}")
else:
    print("No markers detected.")

Marker 'red_pin.png' detected at coordinates (507, 901) with confidence 0.5135186314582825 at scale 0.8


In [8]:
def detect_text_with_coordinates(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use Tesseract to do OCR on the image and get bounding box information
    data = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)

    print("Detected text and their coordinates:")
    
    for i in range(len(data['text'])):
        text = data['text'][i]
        if text.strip():  # Check if text is not empty
            x = data['left'][i]
            y = data['top'][i]
            w = data['width'][i]
            h = data['height'][i]
            adb_coordinates = (x + w // 2, y + h // 2)  # Center of the bounding box

            print(f'Detected text: "{text}"')
            print(f'Coordinates: {adb_coordinates} (x: {x}, y: {y}, width: {w}, height: {h})')

# Call the function
detect_text_with_coordinates(img_path)

Detected text and their coordinates:
Detected text: "5:25@"
Coordinates: (104, 31) (x: 46, y: 12, width: 117, height: 39)
Detected text: "G"
Coordinates: (201, 32) (x: 186, y: 16, width: 31, height: 32)
Detected text: "Ci"
Coordinates: (852, 31) (x: 841, y: 15, width: 23, height: 32)
Detected text: "ain"
Coordinates: (929, 30) (x: 892, y: 14, width: 74, height: 33)
Detected text: "|"
Coordinates: (995, 31) (x: 985, y: 14, width: 20, height: 34)
Detected text: "Amphitheatre"
Coordinates: (779, 445) (x: 700, y: 432, width: 158, height: 27)
Detected text: "Pkwy"
Coordinates: (898, 446) (x: 869, y: 433, width: 59, height: 27)
Detected text: "Brice"
Coordinates: (269, 716) (x: 231, y: 703, width: 76, height: 27)
Detected text: "Rowe"
Coordinates: (362, 716) (x: 321, y: 703, width: 83, height: 27)
Detected text: "Photography"
Coordinates: (517, 720) (x: 418, y: 702, width: 198, height: 36)
Detected text: "Google"
Coordinates: (337, 868) (x: 282, y: 850, width: 110, height: 36)
Detected text:

In [11]:
def detect_text_with_coordinates(image_path, target_text):
    # Load the image
    image = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use Tesseract to do OCR on the image and get bounding box information
    data = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)

    print("Detected text and their coordinates:")
    
    found = False  # Flag to check if target text is found
    
    for i in range(len(data['text'])):
        text = data['text'][i]
        if text.strip():  # Check if text is not empty
            x = data['left'][i]
            y = data['top'][i]
            w = data['width'][i]
            h = data['height'][i]
            adb_coordinates = (x + w // 2, y + h // 2)  # Center of the bounding box

            # Check if the detected text matches the target text
            if text.lower() == target_text.lower():  # Case insensitive comparison
                found = True
                print(f'Target text found: "{text}"')
                print(f'Coordinates: {adb_coordinates} (x: {x}, y: {y}, width: {w}, height: {h})')

    if not found:
        print(f'Target text "{target_text}" not found in the image.')

# Call the function
img_path = 'test.png'  # Replace with your image path
target_text = 'B40'  # Specify the text you want to find
detect_text_with_coordinates(img_path, target_text)


Detected text and their coordinates:
Target text found: "B40"
Coordinates: (435, 864) (x: 406, y: 850, width: 58, height: 28)
